In [11]:
"""import librarys"""
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
import random
from random import randint
import pickle
import csv
from sklearn.model_selection import train_test_split
import tensorflow as tf

import argparse
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU, Activation,SpatialDropout2D
from keras.layers.convolutional import Convolution2D, Cropping2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Lambda
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [20]:
#Hyper Parameters
epoches= 1
batch_size= 32
correction_center=0.31  #0.35links
learning_rate=0.0001
val_size=0.1
dropout_rate1=0.2
dropout_rate2=0.5
print('hyperparameters loaded')

#import data and split samples into training and validation
data_path='/home/chr/Udacity/P3-CarND-Behavioral-Cloning-master/data'

driving_log_path= os.path.join(data_path, 'driving_log3.csv')

samples = []
with open(driving_log_path) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
train_samples, validation_samples = train_test_split(samples, test_size=val_size)
n_train=len(train_samples)
n_valid=len(validation_samples)
print(len(samples))

#derived from udacitys generator example
def generator(samples, batch_size=batch_size):
    n_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, n_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                                                               
                    name='/home/chr/Udacity/P3-CarND-Behavioral-Cloning-master/data/IMG/'+batch_sample[0].split('/')[-1]
                    center_image = cv2.imread(name)
                    center_angle = float(batch_sample[3])-correction_center
                    r=randint(0,1)
                    if r==0:
                        images.append(center_image)
                        angles.append(center_angle)
                    if r==1:
                        images.append(np.fliplr(center_image))
                        angles.append(-center_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)
            
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

# this is an edited model from comma.ai
#(https://github.com/commaai/research/blob/master/train_steering_model.py)


model = Sequential() # needed to add layser sequentially
model.add(Cropping2D(cropping=((70,25),(0,0)), input_shape=(160,320,3))) # cut areas that don't show the road
model.add(Lambda(lambda x: (x/255.0)-0.5)) #normalize pixel values

model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same")) # frist conv2d layer
model.add(ELU()) # Exponential Linear Unit activation function
model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same")) # second conv2d layer
model.add(ELU()) # Exponential Linear Unit activation function
model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same")) # third conv2d layer
model.add(Flatten()) # to combine all found features and not just the local ones like in conv
model.add(Dropout(dropout_rate1)) # 20% of learning data flowing though the net will be lost here
model.add(ELU()) # Exponential Linear Unit activation function
model.add(Dense(512)) # fully connected layer
model.add(Dropout(dropout_rate2)) # 50% of learning data flowing though the net will be lost here
model.add(ELU()) # Exponential Linear Unit activation function
model.add(Dense(1)) # fully connected layer
adam = Adam(lr=learning_rate) # Adaptive moment estimation (Adam) as alternative to stochastic gradient descent
model.compile(optimizer=adam, loss="mse") # loss function is mean squared error

model.fit_generator(train_generator,
                    samples_per_epoch= n_train,
                    validation_data=validation_generator,
                    nb_val_samples=n_valid, 
                    nb_epoch=epoches)

model.save('model3.h5')
print('model3 saved')

hyperparameters loaded
24108
Epoch 1/1
21697/21697 [==============================] - 229s - loss: 0.0822 - val_loss: 0.0711